<style>
  h1, h2, h3, h4, h5, p, ul, li {
    color: #2C475C;
  }
  .output_html {
    color: skyblue;
  }
  hr { height: 2px; color: lightblue; }
</style>

# Spark 101

In [ ]:
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._

221 milliseconds

### First create a dataset using the local `syslog` file

We will 

*  load the file
*  convert each line keeping its size
*  remove the duplicates

For that, we'll use the `sparkContext`, which

* is the driver
* can define job (read inputs, transform, group, etc)
* constructs DAG
* schedules tasks on the cluster

In [ ]:
val dta:RDD[Int] = sparkContext.textFile("/var/log/syslog")
                               .map(_.size)
                               .distinct

MapPartitionsRDD[5] at distinct at <console>:54
 983 milliseconds

**MappedRDD** is actually an instance of `RDD[Int]` that will contain the distinct sizes of the lines.

_Note_: there is NO computations happening! → [see UI](http://localhost:4040/stages/)

-----

### Now we can use the size for fancy operations like grouping per last digit

In [ ]:
val rdd1:RDD[(Int, Iterable[Int])] = dta.groupBy(_ % 10)

ShuffledRDD[7] at groupBy at <console>:54
 404 milliseconds

### But we can also get rid of even sizes (... non trivially...), then _tupling_ with some other computations

In [ ]:
val rdd2 = dta.map(_ + 1)
              .filter(_ % 2 == 0)
              .map(x => (x%10, x*x))

MapPartitionsRDD[10] at map at <console>:56
 360 milliseconds

-----

### We can combine distributed datasets into single ones, by _joining_ them for instance.

In [ ]:
val joined = rdd1.join(rdd2)

MapPartitionsRDD[13] at join at <console>:58
 350 milliseconds

_Note (again)_: still nothing done on the cluster up to here → [see ui](http://localhost:4040/stages/)

-----

#### Now we ask the cluster to do the whole thing: Action

In [ ]:
joined.take(10).toList.mkString("\n")

(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),17956))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),7056))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),4096))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),33856))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),10816))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),5476))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),8836))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),12996))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),26896))
(4,(CompactBuffer(134, 104, 74, 94, 84, 124, 114),15376))
 867 milliseconds

_Note (yeah)_: NOW there were some computations in the cluster → [see stages](http://localhost:4040/stages/) and [see tasks](http://localhost:4040/stages/stage/?id=3&attempt=0)

-----

## But what just happened?

### First Spark created a DAG based on the job definition

In [ ]:
joined.toDebugString

(2) MapPartitionsRDD[13] at join at <console>:58 []
 | MapPartitionsRDD[12] at join at <console>:58 []
 | CoGroupedRDD[11] at join at <console>:58 []
 | ShuffledRDD[7] at groupBy at <console>:54 []
 +-(2) MapPartitionsRDD[6] at groupBy at <console>:54 []
 | MapPartitionsRDD[5] at distinct at <console>:54 []
 | ShuffledRDD[4] at distinct at <console>:54 []
 +-(2) MapPartitionsRDD[3] at distinct at <console>:54 []
 | MapPartitionsRDD[2] at map at <console>:53 []
 | /var/log/syslog MapPartitionsRDD[1] at textFile at <console>:52 []
 | /var/log/syslog HadoopRDD[0] at textFile at <console>:52 []
 +-(2) MapPartitionsRDD[10] at map at <console>:56 []
 | MapPartitionsRDD[9] at filter at <console>:55 []
 | MapPartitionsRDD[8] at map at <console>:54 []
 | MapPartitionsRDD[5] at distinct at <console>:54 []
 | ShuffledRDD[4] at distinct at <console>:54 []
 +-(2) MapPartitionsRDD[3] at distinct at <console>:54 []
 | MapPartitionsRDD[2] at map at <console>:53 []
 | /var/log/syslog MapPartitionsRDD[1] at textFile at <console>:52 []
 | /var/log/syslog HadoopRDD[0] at textFile at <console>:52 []
 384 milliseconds

### Then it scheduled it to the executors in the cluster <small>only one when running in local mode<small>

We can check the <strong>Total tasks</strong> activity in the [UI](http://localhost:4040/executors/)

-------

## Now we will prepare the dataset and then using it several times

So we'll read a file about stock price per day, so let's create a type holding relevant data.

In [ ]:
case class Quote(stock:String, date:String, price:Double) extends java.io.Serializable

194 milliseconds

The file will contain lines like:
``` 
ASTE,2011-12-06,33.93
ASTE,2012-03-14,36.84
```

Let's download the data first

In [ ]:
import sys.process._
"mkdir -p /tmp/data"!!

if (!new java.io.File("/tmp/data/closes.csv").exists)
  "wget https://s3-eu-west-1.amazonaws.com/spark-notebook-data/closes.csv -O /tmp/data/closes.csv"!!

1 minute 30 seconds 87 milliseconds

In [ ]:
:sh du -h /tmp/data/closes.csv

174M	/tmp/data/closes.csv
 
 595 milliseconds

In [ ]:
val closes:RDD[Quote] = sparkContext.textFile("/tmp/data/closes.csv")
                                   .map(_.split(",").toList)
                                   .map{ case s::d::p::Nil => Quote(s, d, p.toDouble)}

MapPartitionsRDD[17] at map at <console>:62
 682 milliseconds

We have date, so we can group stock prices per day

In [ ]:
val byDate = closes.keyBy(_.date)

Now we can compute the minimum stocks per date

In [ ]:
def minByDate = byDate.combineByKey[(String, Double)](                                                                                           // `def` to force spark recomputing... otherwise it's smart enough to reuse previous RDDs...
  (x:Quote) => (x.stock, x.price), 
  (d:(String, Double), l:Quote) => if (d._2 < l.price) d else (l.stock, l.price),
  (d1:(String, Double), d2:(String, Double)) => if (d1._2 < d2._2) d1 else d2
)

In [ ]:
<pre>{minByDate.take(2).toList.mkString("\n")}</pre>

It took ~2 seconds (in local[8] and 24G of RAM)

In [ ]:
<pre>{minByDate.take(2).toList.mkString("\n")}</pre>

Once again.... 2 seconds!!!

#### Solution: caching!

In [ ]:
val maxByDate2 = byDate.combineByKey[(String, Double)](
  (x:Quote) => (x.stock, x.price), 
  (d:(String, Double), l:Quote) => if (d._2 > l.price) d else (l.stock, l.price),
  (d1:(String, Double), d2:(String, Double)) => if (d1._2 > d2._2) d1 else d2
)

maxByDate2.cache()                                                                                                               // okay.... not really needed since Spark is smart enough in this case -_-"

Ask some data

In [ ]:
<pre>{maxByDate2.take(2).toList.mkString("\n")}</pre>

**Go to [UI](http://localhost:4040/storage/)**

In [ ]:
<pre>{maxByDate2.take(2).toList.mkString("\n")}</pre>


     while compiling: <console>
        during phase: global=icode, atPhase=parser
     library version: version 2.10.4
    compiler version: version 2.10.4
  reconstructed args: -classpath /home/Sources/noootsab/spark-notebook/target/scala-2.10/classes:/home/Sources/noootsab/spark-notebook/modules/tachyon/target/scala-2.10/classes:/home/Sources/noootsab/spark-notebook/modules/subprocess/target/scala-2.10/classes:/home/Sources/noootsab/spark-notebook/modules/observable/target/scala-2.10/classes:/home/Sources/noootsab/spark-notebook/modules/common/target/scala-2.10/classes:/home/Sources/noootsab/spark-notebook/modules/spark/target/scala-2.10/classes:/home/Sources/noootsab/spark-notebook/modules/kernel/target/scala-2.10/classes:/home/noootsab/.sbt/boot/scala-2.10.4/lib/scala-compiler.jar:/home/noootsab/.sbt/boot/scala-2.10.4/lib/scala-library.jar:/home/noootsab/.sbt/boot/scala-2.10.4/lib/scala-reflect.jar:/home/noootsab/.ivy2/cache/com.google.guava/guava/bundles/guava-14.0.1.jar:/home/n

**BLAZING FAST** => Reuses the cache!